In [11]:
!pip install transformers
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 1.3 MB 31.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=71fa1779b74dca4c5f673e4f7a97a40bfc742fd5fcbf5b5fdd5e25879ecf9c5f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [65]:
import numpy as np
import math
import re
import pandas as pd
import random
from transformers import BertTokenizer, BertForMaskedLM
from transformers import BertModel
from transformers import BertForSequenceClassification
import torch

In [66]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
data = pd.DataFrame([['REPORT SPEED','SLOW MOVING'], ['MAINTAIN PRESENT SPEED','FAST MOVING'],['RESUME NORMAL SPEED','CLOSING'],['REDUCE TO MINIMUM APPROACH SPEED','CLIMBING'],['SPEED MAXIMUM','TURN LEFT'],['RESUME PUBLISHED SPEED','OVERTAKING']], columns=['speed command','Traffic'])
data.to_csv('data.csv')


In [82]:
print(data)

                      speed command      Traffic
0                      REPORT SPEED  SLOW MOVING
1            MAINTAIN PRESENT SPEED  FAST MOVING
2               RESUME NORMAL SPEED      CLOSING
3  REDUCE TO MINIMUM APPROACH SPEED     CLIMBING
4                     SPEED MAXIMUM    TURN LEFT
5            RESUME PUBLISHED SPEED   OVERTAKING


In [83]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [84]:
# Function returning vector reperesentation of a document
def get_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(5)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        print("done")
        return np.array(embeddings)
        return np.mean(embeddings, axis=0)
data['vector']=data['speed command'].apply(lambda x :get_embedding(x))
data['vector1']=data['Traffic'].apply(lambda x :get_embedding(x))

done
done
done
done
done
done
done
done
done
done
done
done


In [85]:
data

,speed command,Traffic,vector,vector1
0,REPORT SPEED,SLOW MOVING,"[[-0.28489488, -0.61145985, 2.4605138, 0.99001...","[[0.1515865, -0.42877954, 1.2250288, 0.2561459..."
1,MAINTAIN PRESENT SPEED,FAST MOVING,"[[-0.22803828, -0.4560589, 2.4652727, 0.890132...","[[-0.28566116, -0.8084894, 1.494205, 0.7216701..."
2,RESUME NORMAL SPEED,CLOSING,"[[-0.030971806, -0.9375227, 1.943133, 0.897214...","[[-0.10938317, 0.15788686, 2.5348787, -0.01871..."
3,REDUCE TO MINIMUM APPROACH SPEED,CLIMBING,"[[-0.5929593, -0.15124826, 1.3369446, 0.613844...","[[0.26563948, -0.45666885, 2.1009676, 0.269733..."
4,SPEED MAXIMUM,TURN LEFT,"[[-0.45263094, -0.812656, 1.8271861, 0.9092465...","[[-0.07302494, -0.14070737, 1.5531414, -0.1440..."
5,RESUME PUBLISHED SPEED,OVERTAKING,"[[-0.2296048, -0.37632227, 2.136449, 0.7627664...","[[-0.42281353, -0.7720194, 2.0352926, 0.842180..."


In [94]:
from sklearn.metrics.pairwise import cosine_similarity
array_vec_1 = data['vector']
array_vec_2 = data['vector1']
query='NORMAL SPEED'
array_vec_3 = get_embedding(query)
for i in array_vec_1:
  print("speed command and query",cosine_similarity(i, array_vec_3))
for i in array_vec_2:
  print("Traffic and query",cosine_similarity(i, array_vec_3))




done
speed command and query [[0.65832347]]
speed command and query [[0.7551709]]
speed command and query [[0.90465224]]
speed command and query [[0.50509274]]
speed command and query [[0.6993253]]
speed command and query [[0.57452166]]
Traffic and query [[0.6365405]]
Traffic and query [[0.5915581]]
Traffic and query [[0.36166006]]
Traffic and query [[0.4368865]]
Traffic and query [[0.37966734]]
Traffic and query [[0.5106064]]
